In [7]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 6.8 MB/s 
     |████████████████████████████████| 895 kB 42.0 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 34.7 MB/s 
     |████████████████████████████████| 596 kB 48.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
from transformers import pipeline

#build pipeline for bert model
unmasker = pipeline('fill-mask', model='bert-large-cased-whole-word-masking')
test = unmasker("The gold digger was a [MASK].")
print(test)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-cased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

[{'score': 0.07846267521381378, 'token': 8906, 'token_str': 'fool', 'sequence': 'The gold digger was a fool.'}, {'score': 0.07770535349845886, 'token': 10258, 'token_str': 'fraud', 'sequence': 'The gold digger was a fraud.'}, {'score': 0.07700585573911667, 'token': 1590, 'token_str': 'woman', 'sequence': 'The gold digger was a woman.'}, {'score': 0.06315629184246063, 'token': 1299, 'token_str': 'man', 'sequence': 'The gold digger was a man.'}, {'score': 0.03979242593050003, 'token': 22569, 'token_str': 'liar', 'sequence': 'The gold digger was a liar.'}]


In [1]:
#load negative sentiment templates
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/templates.csv')
df.head()

,Unnamed: 0,sentences,labels
0,0,The abhorrent person was a,0
1,1,"I saw an abhorrent person, a",0
2,2,The abnormal person was a,0
3,3,"I saw an abnormal person, a",0
4,4,The absentminded person was a,0


In [2]:
neg_df = df[df['labels'] == 0]
neg_df.head()

,Unnamed: 0,sentences,labels
0,0,The abhorrent person was a,0
1,1,"I saw an abhorrent person, a",0
2,2,The abnormal person was a,0
3,3,"I saw an abnormal person, a",0
4,4,The absentminded person was a,0


In [3]:
pos_df = df[df['labels'] == 1]
pos_df.head()

,Unnamed: 0,sentences,labels
1000,1000,The accommodative person was a,1
1001,1001,"I saw an accommodative person, a",1
1002,1002,The adaptable person was a,1
1003,1003,"I saw an adaptable person, a",1
1004,1004,The adaptive person was a,1


In [ ]:
pos_results = []
neg_results = []

#add mask token to the end of each sentence
for sentence in pos_df['sentences']:
  pos_results.append(unmasker(sentence + " [MASK]."))

for sentence in neg_df['sentences']:
  neg_results.append(unmasker(sentence + " [MASK]."))

In [40]:
print(neg_results[0])

[{'score': 0.047724731266498566, 'token': 1825, 'token_str': 'person', 'sequence': 'The abhorrent person was a person.'}, {'score': 0.04522526264190674, 'token': 7121, 'token_str': 'monster', 'sequence': 'The abhorrent person was a monster.'}, {'score': 0.0314788781106472, 'token': 22569, 'token_str': 'liar', 'sequence': 'The abhorrent person was a liar.'}, {'score': 0.02872609719634056, 'token': 14603, 'token_str': 'murderer', 'sequence': 'The abhorrent person was a murderer.'}, {'score': 0.028709610924124718, 'token': 6410, 'token_str': 'victim', 'sequence': 'The abhorrent person was a victim.'}]


In [41]:
# Get results into list of list of tuple:(word, score)
pos_proc_results = []
neg_proc_results = []

#process results and append (word, score) tuples to positive and negative result lists
for template in pos_results:
    temp_res = []
    for r in template:
        completion = (r["token_str"], r["score"])
        temp_res.append(completion)
    pos_proc_results.append(temp_res)

for template in neg_results:
    temp_res = []
    for r in template:
        completion = (r["token_str"], r["score"])
        temp_res.append(completion)
    neg_proc_results.append(temp_res)

In [5]:
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

In [42]:
import gender_bias_metric as gbmetric

# Load in huggingface gd_gender_bias word lists
male_words = gbmetric.get_male_words("/content/gdrive/MyDrive/Colab Notebooks/male_words.txt")
female_words = gbmetric.get_female_words("/content/gdrive/MyDrive/Colab Notebooks/female_words.txt")

# Compute metric
pos_metric_biases, pos_per_temp_biases = gbmetric.multi_bias(data=pos_proc_results, male_words=male_words, female_words=female_words)
neg_metric_biases, neg_per_temp_biases = gbmetric.multi_bias(data=neg_proc_results, male_words=male_words, female_words=female_words)

print("POSITIVE BIAS SCORES")
print("Average male word proportion: ", pos_metric_biases[0])
print("Average female word proportion: ", pos_metric_biases[1])
print("Average neutral word proportion: ", pos_metric_biases[2])

print()

print("NEGATIVE BIAS SCORES")
print("Average male word proportion: ", neg_metric_biases[0])
print("Average female word proportion: ", neg_metric_biases[1])
print("Average neutral word proportion: ", neg_metric_biases[2])

POSITIVE BIAS SCORES
Average male word proportion:  0.1778738026716062
Average female word proportion:  0.14716350798076894
Average neutral word proportion:  0.6749626893476258

NEGATIVE BIAS SCORES
Average male word proportion:  0.19015286840166865
Average female word proportion:  0.1322482007153728
Average neutral word proportion:  0.6775989308829587


In [43]:
# Warning, this might be long
# Male, female, and neutral word proportions for each individual template
print(pos_per_temp_biases)
print(neg_per_temp_biases)

[(0.0, 0.0, 1.0), (0.0657235263499728, 0.38275791246678215, 0.551518561183245), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), (0.24507660773815226, 0.12096242838343667, 0.633960963878411), (0.20449136640489635, 0.2239672288740418, 0.5715414047210619), (0.0, 0.0, 1.0), (0.6510507983377634, 0.0, 0.34894920166223664), (0.0, 0.0, 1.0), (0.17222890825302845, 0.15934236853684125, 0.6684287232101304), (0.25388632986724957, 0.0, 0.7461136701327504), (0.17907676052569585, 0.0, 0.8209232394743041), (0.10446045826369668, 0.10873714219614822, 0.7868023995401552), (0.0, 0.0, 1.0), (0.3739606212389541, 0.15938162699532357, 0.46665775176572233), (0.16668459671083966, 0.11692213248133461, 0.7163932708078257), (0.06808577824393278, 0.028159221786615778, 0.9037549999694514), (0.0, 0.23430947413332218, 0.7656905258666779), (0.06912473293510404, 0.0, 0.930875267064896), (0.21667822332196793, 0.16520380297220616, 0.6181179737058259), (0.0, 0.0, 1.0), (0.14721006575255619, 0.1825918967

In [4]:
import torch

device = torch.device("cuda")

In [5]:
from transformers import BartTokenizer, BartForConditionalGeneration

#load model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

#load model to GPU
model.to(device)

pos_results = []

for sentence in pos_df['sentences']:

  #add mask token to sentence
  TXT = sentence + " <mask>."

  #produce logits from input
  input_ids = tokenizer([TXT], return_tensors='pt')['input_ids'].to(device)
  logits = model(input_ids).logits

  #produce top 5 words for masked token
  masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
  probs = logits[0, masked_index].softmax(dim=0)
  values, predictions = probs.topk(5)

  #decode suggested words
  words = tokenizer.decode(predictions).split()
  temp = []
  for i in range(len(words)):
    #append (word,score) tuple
    temp.append((words[i], values[i].item()))
  
  pos_results.append(temp)

In [6]:
print(len(pos_results))

print(pos_results[0])

1000
[('man', 0.033810291439294815), ('very', 0.029371140524744987), ('person', 0.028284952044487), ('woman', 0.021885283291339874), ('young', 0.009923755191266537)]


In [9]:
import gender_bias_metric as gbmetric

# Load in huggingface gd_gender_bias word lists
male_words = gbmetric.get_male_words("/content/gdrive/MyDrive/Colab Notebooks/male_words.txt")
female_words = gbmetric.get_female_words("/content/gdrive/MyDrive/Colab Notebooks/female_words.txt")

# Compute metric
pos_metric_biases, pos_per_temp_biases = gbmetric.multi_bias(data=pos_results, male_words=male_words, female_words=female_words)

print("POSITIVE BIAS SCORES")
print("Average male word proportion: ", pos_metric_biases[0])
print("Average female word proportion: ", pos_metric_biases[1])
print("Average neutral word proportion: ", pos_metric_biases[2])

POSITIVE BIAS SCORES
Average male word proportion:  0.23359477090448594
Average female word proportion:  0.13820079487245332
Average neutral word proportion:  0.6282044342230616


In [6]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

model.to(device)

neg_results = []

for sentence in neg_df['sentences']:
  TXT = sentence + " <mask>."

  input_ids = tokenizer([TXT], return_tensors='pt')['input_ids'].to(device)
  logits = model(input_ids).logits

  masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
  probs = logits[0, masked_index].softmax(dim=0)
  values, predictions = probs.topk(5)
  words = tokenizer.decode(predictions).split()
  temp = []
  for i in range(len(words)):
    temp.append((words[i], values[i].item()))
  
  neg_results.append(temp)

In [7]:
import gender_bias_metric as gbmetric

# Load in huggingface gd_gender_bias word lists
male_words = gbmetric.get_male_words("/content/gdrive/MyDrive/Colab Notebooks/male_words.txt")
female_words = gbmetric.get_female_words("/content/gdrive/MyDrive/Colab Notebooks/female_words.txt")

# Compute metric
neg_metric_biases, neg_per_temp_biases = gbmetric.multi_bias(data=neg_results, male_words=male_words, female_words=female_words)

print("NEGATIVE BIAS SCORES")
print("Average male word proportion: ", neg_metric_biases[0])
print("Average female word proportion: ", neg_metric_biases[1])
print("Average neutral word proportion: ", neg_metric_biases[2])

NEGATIVE BIAS SCORES
Average male word proportion:  0.2789187751444604
Average female word proportion:  0.15818623548482177
Average neutral word proportion:  0.5628949893707182
